## Librerias

In [1]:
# Manipulación de datos
import pandas as pd
import numpy as np
import json
from pandas import json_normalize

# Conexión con la base de datos
import psycopg2
import pymysql
from sqlalchemy import create_engine

## Conexión con la base de datos

In [2]:
conn = psycopg2.connect(
    database = "postgres",
    user = "postgres",
    host = "localhost", 
    password = "example", 
    port = "5432"
    )

## Load data

In [3]:
with open('mpd.slice.1000-1999.json') as f:
   data = json.load(f)

df = json_normalize(data['playlists'])

## Unificacion de todo en una tabla

In [4]:
# se le agregan indices a las playlists

df = df.reset_index()

#### Creación de df canciones, con las canciones y si sk_playlist

In [5]:
canciones = pd.DataFrame(df.tracks[0])
canciones["sk_playlist"] = df.index[0]

for i in range(1, len(df)):
    new = pd.DataFrame(df.tracks[i])
    new["sk_playlist"] = df.index[i]
    
    canciones = pd.concat([canciones,new])

#### Creacion de indices para cada tabla

In [6]:
# sk_track
id_tracks = canciones["track_uri"].unique()
id_tracks = pd.DataFrame(id_tracks, columns=["track_uri"]).reset_index()
id_tracks = id_tracks.rename({"index":"sk_track"},axis=1)


In [7]:
# sk_album
id_album = canciones["album_uri"].unique()
id_album = pd.DataFrame(id_album, columns=["album_uri"]).reset_index()
id_album = id_album.rename({"index":"sk_album"},axis=1)


In [8]:
# sk_album
id_artist = canciones["artist_uri"].unique()
id_artist  = pd.DataFrame(id_artist, columns=["artist_uri"]).reset_index()
id_artist = id_artist.rename({"index":"sk_artist"},axis=1)


#### Union de los indices con sus tablas

In [9]:
## join con sk_tracks 7

canciones2 = canciones.copy()
canciones2 = canciones2.merge(id_tracks, left_on="track_uri", right_on="track_uri", how="inner")

## join con sk_album 
canciones2 = canciones2.merge(id_album, left_on="album_uri", right_on="album_uri", how="inner")

## join con sk_artist 
canciones2 = canciones2.merge(id_artist, left_on="artist_uri", right_on="artist_uri", how="inner")


### Creacion de la tabla final

In [10]:
## join de la tabla de plalist con la tabla original 
playlist_completa = df.copy()
playlist_completa = playlist_completa.merge(canciones2, left_on="index", right_on="sk_playlist", how="inner")
playlist_completa["sk_playlist_track"] = range(0, len(playlist_completa))

print(playlist_completa.shape)
playlist_completa.head(3)

(66622, 26)


,index,name,collaborative,pid,modified_at,num_tracks,num_albums,num_followers,tracks,num_edits,...,artist_uri,track_name,album_uri,duration_ms_y,album_name,sk_playlist,sk_track,sk_album,sk_artist,sk_playlist_track
0,0,disney,false,1000,1457827200,189,16,1,"[{'pos': 0, 'artist_name': 'Original Broadway ...",4,...,spotify:artist:3TymzPhJTMyupk7P5xkahM,Fathoms Below - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,154506,The Little Mermaid: Original Broadway Cast Rec...,0,0,0,0,0
1,0,disney,false,1000,1457827200,189,16,1,"[{'pos': 0, 'artist_name': 'Original Broadway ...",4,...,spotify:artist:3TymzPhJTMyupk7P5xkahM,Daughters Of Triton - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,79066,The Little Mermaid: Original Broadway Cast Rec...,0,1,0,0,1
2,0,disney,false,1000,1457827200,189,16,1,"[{'pos': 0, 'artist_name': 'Original Broadway ...",4,...,spotify:artist:3TymzPhJTMyupk7P5xkahM,The World Above - Broadway Cast Recording,spotify:album:3ULJeOMgroG27dpn27MDfS,94600,The Little Mermaid: Original Broadway Cast Rec...,0,2,0,0,2


## Extración de las tablas

In [11]:
# Extracción de d_albums

d_album = playlist_completa[["sk_album", "album_uri", "album_name"]].drop_duplicates()
d_album.shape

(20212, 3)

In [12]:
# Extraccion de d_artists 

d_artists = playlist_completa[["sk_artist", "artist_uri", "artist_name"]].drop_duplicates()
d_artists.shape

(10303, 3)

In [13]:
# Extracción de d_playlists

d_playlists = playlist_completa[["sk_playlist", "pid", "name", "collaborative", "modified_at", "num_followers", "num_edits"]].drop_duplicates()
d_playlists.shape

(1000, 7)

In [14]:
# Extracción de d_tracks

d_tracks = playlist_completa[["sk_track", "track_uri", "track_name", "duration_ms_y"]].drop_duplicates()
d_tracks.shape

(36175, 4)

In [15]:
# Extracción de d_playlist_tracks

d_playlist_tracks = playlist_completa[[
    "sk_playlist_track", 
    "sk_playlist",  
    "sk_artist",
    "sk_album", 
    "sk_track",
    "pos"
]].drop_duplicates()
d_playlist_tracks.shape

(66622, 6)

## Carga a la base de datos 

In [16]:
# Inserción de datos en la tabla d_albums 
tuples = [tuple(x) for x in d_album.to_numpy()]

query = "INSERT INTO " + "d_albums" + "(" + "sk_id, bk_id, name" + ")" + "VALUES(" + "%s, %s, %s" + ");"

cursor = conn.cursor()

try: 
    cursor.executemany(query, tuples)
    conn.commit()
    print("La carga de datos fue un exito...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Error al cargar los datos...")

cursor.close()


La carga de datos fue un exito...


In [17]:
# Inserción de datos en la tabla d_artists 
tuples = [tuple(x) for x in d_artists.to_numpy()]

query = "INSERT INTO " + "d_artists" + "(" + "sk_id, bk_id, name" + ")" + "VALUES(" + "%s, %s, %s" + ");"

cursor = conn.cursor()

try: 
    cursor.executemany(query, tuples)
    conn.commit()
    print("La carga de datos fue un exito...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Error al cargar los datos...")

cursor.close()

La carga de datos fue un exito...


In [18]:
# Inserción de datos en la tabla d_playlists 
tuples = [tuple(x) for x in d_playlists.to_numpy()]

query = "INSERT INTO " + "d_playlists" + "(" + "sk_id, bk_id, name, collaborative, last_modified_at, num_followers, num_edits" + ")" + "VALUES(" + "%s, %s, %s, %s, to_timestamp(%s), %s, %s" + ");"

cursor = conn.cursor()

try: 
    cursor.executemany(query, tuples)
    conn.commit()
    print("La carga de datos fue un exito...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Error al cargar los datos...")

cursor.close()

La carga de datos fue un exito...


In [19]:
# Inserción de datos en la tabla d_tracks 
tuples = [tuple(x) for x in d_tracks.to_numpy()]

query = "INSERT INTO " + "d_tracks" + "(" + "sk_id, bk_id, name, duration" + ")" + "VALUES(" + "%s, %s, %s, %s" + ");"

cursor = conn.cursor()

try: 
    cursor.executemany(query, tuples)
    conn.commit()
    print("La carga de datos fue un exito...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Error al cargar los datos...")

cursor.close()

La carga de datos fue un exito...


In [20]:
d_playlist_tracks = d_playlist_tracks.convert_dtypes()

In [21]:
# Inserción de datos en la tabla f_playlist_tracks
tuples = [tuple(x) for x in d_playlist_tracks.to_numpy()]

columnas = "id, sk_playlist, sk_artist, sk_album, sk_track, track_position"
formato = "%s, %s, %s, %s, %s, %s"
query = "INSERT INTO " + "f_playlist_tracks" + "(" + columnas + ")" + "VALUES(" + formato + ");"

cursor = conn.cursor()

try: 
    cursor.executemany(query, tuples)
    conn.commit()
    print("La carga de datos fue un exito...")

except (Exception, psycopg2.DatabaseError) as error:
    print("Error: ", error)
    conn.rollback()
    print("Error al cargar los datos...")

cursor.close()

La carga de datos fue un exito...


## Consulta de las tablas

In [22]:
# tabla d_albums
cursor = conn.cursor()

cursor.execute("select * from d_albums")
data = cursor.fetchall()

cursor.close()
pd.DataFrame(data, columns=["sk_id", "bk_id", "name"])

,sk_id,bk_id,name
0,0,spotify:album:3ULJeOMgroG27dpn27MDfS,The Little Mermaid: Original Broadway Cast Rec...
1,1,spotify:album:4aAwvCRNJIqiUGVEjieWv6,Little Mermaid
2,2,spotify:album:5wqx6PBsow0KIQrnu9xd6c,Aladdin Original Broadway Cast Recording
3,4,spotify:album:6DnTzVKb79zEtICWeIB2El,Beauty And The Beast: The Broadway Musical
4,6,spotify:album:3O5p9VNddbwvqWTdYKEqV5,Beauty and the Beast
...,...,...,...
20207,20202,spotify:album:2BjRmYDj4YQ0ywxaV5VISh,Follow Through
20208,20209,spotify:album:3nta4nhqWoWjc6LmHIB0kT,Verities & Balderdash
20209,20208,spotify:album:77yXUMeh96foFhIR2aK3Fb,De Cine
20210,20210,spotify:album:5G6pS4ES22Np0hehqJYIct,Title Of Record


In [23]:
# tabla d_artists 
cursor = conn.cursor()

cursor.execute("select * from d_artists")
data = cursor.fetchall()

cursor.close()
pd.DataFrame(data, columns=["sk_id", "bk_id", "name"])

,sk_id,bk_id,name
0,0,spotify:artist:3TymzPhJTMyupk7P5xkahM,Original Broadway Cast - The Little Mermaid
1,1,spotify:artist:1j4a3bBWYNjecFqxcjRtr5,Sierra Boggess as Ariel
2,2,spotify:artist:5sy77gt4bfsLcSQ8GIe4ZZ,Alan Menken
3,3,spotify:artist:1RJ9qwSsCfnOcTXBSqotNk,Ariel
4,4,spotify:artist:6Id8rcDNyBXPcgKQVfQ8rX,Samuel E. Wright
...,...,...,...
10298,10298,spotify:artist:3tM6SwRysCaIK0kW4h0mVP,Spose
10299,10299,spotify:artist:4qUFkkArfii3qNG8TnbPfc,Masked Intruder
10300,10300,spotify:artist:7C64wNX3howEFZjAYRKsfP,The Cataracs
10301,10301,spotify:artist:6pxwzi0xDny2rMI1KJFZoi,Young T.H.U.G.


In [24]:
# tabla d_playlists
cursor = conn.cursor()

cursor.execute("select * from d_playlists")
data = cursor.fetchall()

cursor.close()
pd.DataFrame(data, columns=["sk_id", "bk_id", "name", "collaborative", "last_modified_at", "num_followers", "num_edits"])

,sk_id,bk_id,name,collaborative,last_modified_at,num_followers,num_edits
0,0,1000,disney,False,2016-03-13,1,4
1,1,1001,Indie Electro,False,2014-12-06,2,2
2,2,1002,jack & jack,False,2016-06-09,1,3
3,3,1003,vibes,False,2017-06-26,2,91
4,4,1004,Indie,False,2017-06-28,1,74
...,...,...,...,...,...,...,...
995,995,1995,woo,False,2017-10-27,2,13
996,996,1996,NEW YEARS,False,2015-01-01,1,3
997,997,1997,JESUS,False,2017-10-19,2,8
998,998,1998,yep,False,2015-11-23,1,3


In [25]:
# tabla d_tracks
cursor = conn.cursor()

cursor.execute("select * from d_tracks")
data = cursor.fetchall()

cursor.close()
pd.DataFrame(data, columns=["sk_id", "bk_id", "name", "duration"])

,sk_id,bk_id,name,duration
0,0,spotify:track:5IbCV9Icebx8rR6wAp5hhP,Fathoms Below - Broadway Cast Recording,154506
1,1,spotify:track:6rKVAvjHcxAzZ1BHtwh5yC,Daughters Of Triton - Broadway Cast Recording,79066
2,2,spotify:track:6Jlkb1Wh08RYHstWScsTvg,The World Above - Broadway Cast Recording,94600
3,3,spotify:track:0XhC8bfStML9ygBmfOt1JJ,Human Stuff - Broadway Cast Recording,151480
4,4,spotify:track:0ABxAcsRWlqckkyONsfP67,I Want the Good Times Back - Broadway Cast Rec...,297920
...,...,...,...,...
36170,36172,spotify:track:1bPRhOz3nOfEIoLjB1YYT3,No More,225493
36171,36173,spotify:track:1o4Pf2GIMDCD7ifIM2yI77,Last to Know,207466
36172,36171,spotify:track:3bq8WiQrEXMwPAibYs7tpu,Nobody Praying For Me,197800
36173,36170,spotify:track:2c5wml8wsBnPfNSbCZ574T,Take a Picture,359520


In [26]:
# tabla f_playlist_tracks
cursor = conn.cursor()

cursor.execute("select * from f_playlist_tracks")
data = cursor.fetchall()

cursor.close()
pd.DataFrame(data, columns=["id", "sk_playlist", "sk_artist", "sk_album", "sk_track", "track_position"])

,id,sk_playlist,sk_artist,sk_album,sk_track,track_position
0,0,0,0,0,0,0
1,1,0,0,0,1,1
2,2,0,0,0,2,2
3,3,0,0,0,3,3
4,4,0,0,0,4,4
...,...,...,...,...,...,...
66617,66617,999,2573,4780,7256,11
66618,66618,999,4604,12699,21523,4
66619,66619,999,6410,20210,36170,5
66620,66620,999,6674,12335,20780,7


In [27]:
conn.close()